## Level 1 Requirement Flowdown

### L1-4 Regional Fluxes (Target Mode)

> For representative regions² (90 km × 90 km) Carbon-I shall obtain column averaged mixing ratios with a standard error ≤8 ppm CH4, ≤1.6 ppm CO2, and ≤10 ppb CO once averaged to a 1 km × 1 km grid for our reference scenario4 with a revisit capability ≤7 days. 


²: Representative regions are those that best exemplify major emission sectors—wetlands, agriculture, and waste—where emission uncertainty is high and process-based understanding remains limited.

---

Let's codify these numbers

In [ ]:
using Pkg
# Activate the environment for the project
# This is necessary to ensure that the correct versions of packages are used
Pkg.activate("../.."); # Actives the environment at the root of the project
# Using our Carbon-I and vSmartMOM packages (developed by our team)
using CarbonI, vSmartMOM, Printf
using CairoMakie, DelimitedFiles, Statistics, Interpolations, NCDatasets, InstrumentOperator, LinearAlgebra
using Unitful, UnitfulEquivalences
include(joinpath("../../src/Plots", "CI_colorsNew.jl"));
include(joinpath(dirname(pathof(CarbonI)), "readSun.jl"))
include(joinpath(dirname(pathof(CarbonI)), "Requirements", "common.jl"))

set_theme!(theme_ggplot2())


  Activating project at `~/code/gitHub/CarbonI`


In [ ]:
# Define numbers from L1 Requirements
regional_mode_ch4_error = 12.0            # in ppb
regional_mode_co2_error =  2.0            # in ppm
regional_mode_co_error  = 10.0            # in ppb

# Regional Mode (L1-5)
regional_mode_grid_resolution = [1000, 1000] # in m
regional_mode_revisit_days = 7  # in days
regional_mode_region_size = [100, 100]  # in km
# Account for median pixel size enhancements for GSD requirements at off-nadir angles

offNadir_factor = 2 # Account for GSD increase at 30/30 degree off nadir along and across track
regional_mode_gsd_req = [50.0*offNadir_factor, 50.0 ]
regional_mode_gsd_cbe = [34.5*offNadir_factor, 30.3 ]

# Reference Background VMRs
background_vmr_ch4 = 1900.0   # in ppb
background_vmr_co2 =  420.0   # in ppm
background_vmr_n2o =  330.0   # in ppb

Now we can compute the potential number of footprints in the regional grid, i.e.
$$
N_{max} = \frac{GSD_x \times GSD_y}{Grid_x \times Grid_y}
$$:

In [ ]:
# Maximum number of global ground pixels in the 12km box
# Req.
N_max_req = prod(regional_mode_grid_resolution)/prod(regional_mode_gsd_req);
# At CBE
N_max_cbe = prod(regional_mode_grid_resolution)/prod(regional_mode_gsd_cbe);

For regional mode, we select targets based on optimal visibility, defining a stricter criterion for cloud-free scenes. This demonstrates achievable performance under clear conditions outside the humid tropics, where surface albedo is typically higher. For tropical regions, achieving similar precision would require using a coarser inversion grid—an approach we can also illustrate.

In [ ]:
# Number of cloud-free  ground pixels in the 1km box (here)
# Req.
f_cloud = 1.0
N_eff_req = N_max_req * f_cloud;
# At CBE
N_eff_cbe = N_max_cbe * f_cloud;
@printf("Number of cloud-free Pixel in Required GSD: %.3g\n", N_eff_req)
@printf("Number of cloud-free Pixel in CBE GSD: %.3g\n", N_eff_cbe)

We can now derive the requirement for $\sigma_{s}^{rel}$, as:
$$
\sigma_s^{rel} < \underbrace{\sigma_{agg}^{rel}}_{Required} \times \sqrt{N_{eff}}  
$$

In [ ]:
sigma_rel_agg_req      =  min(regional_mode_ch4_error/background_vmr_ch4 ,  regional_mode_co2_error/background_vmr_co2)  # This should effectively be the same relative error for CO2 and CH4
sigma_rel_SingleSounding_req  =  sigma_rel_agg_req * sqrt(N_eff_req)  
sigma_rel_SingleSounding_cbe  =  sigma_rel_agg_req * sqrt(N_eff_cbe)
@printf("Required Single Measurement Precision for N₂O in percent for req. GSD : %.3g\n", 100*sigma_rel_SingleSounding_req)
@printf("Required Single Measurement Precision for N₂O in percent for CBE specs: %.3g\n", 100*sigma_rel_SingleSounding_cbe)

In [ ]:
@show 100*sigma_rel_agg_req

Just to show that the CH4 or CO2 precisions hardly matter for the relative errors:
$$
\sigma_{\mathrm{rel}}(R)
\;=\;\sqrt{\;\sigma_{\mathrm{rel}}^2\!\bigl(\Omega_{\mathrm{CH_4,CO_2}}\bigr)
+\;\sigma_{\mathrm{rel}}^2\!\bigl(\Omega_{\mathrm{N_2O}}\bigr)\,},
$$

In [ ]:
σ_n2o = 3.5 # Error in %
σ_ghg = 0.5 # Error in % (CO2 and CH4, even smaller than that)
σ_rel = sqrt(σ_n2o^2 + σ_ghg^2)
@printf("Error Enhancement factor if also considering GHG noise: %.3g\n", σ_rel / σ_n2o)
ampl = σ_rel / σ_n2o

Now we have to show what instrument specs we require to get to that precision. 
--> We could bypass SNR and just use our tropical reference scene (with spectrally varying albedo) and compute sigma as function of total system efficiency (i.e. optical throughput and Qe)

In [ ]:
# Load Reference scenario (Typical scene)
scenario = CarbonI.reference_scenario()

req_specs = CarbonI.build_instrument("Requirement") 
cbe_specs = CarbonI.build_instrument("CBE") 

# Create required Instrument model 
ins_req = InstrumentOperator.createGratingNoiseModel(req_specs.ET, req_specs.Pitch, 
		req_specs.FPA_quantum_efficiency, req_specs.bench_efficiency, 
		req_specs.Fnumber, 2*req_specs.SSI, 
		(req_specs.readout_noise), req_specs.dark_current); 

# Create required Instrument model
ins_cbe = InstrumentOperator.createGratingNoiseModel(cbe_specs.ET, cbe_specs.Pitch, 
		cbe_specs.FPA_quantum_efficiency, cbe_specs.bench_efficiency, 
		cbe_specs.Fnumber, 2*cbe_specs.SSI, 
		(cbe_specs.readout_noise), cbe_specs.dark_current); 

soil_req, x_req, solarIrr_req, σ_matrix_req, profile_req, h_req, Sₐ_req = setup_data(scenario, req_specs)
soil_cbe, x_cbe, solarIrr_cbe, σ_matrix_cbe, profile_cbe, h_cbe, Sₐ_cbe = setup_data(scenario, cbe_specs);

In [ ]:
refl_req   = scenario.surface_albedo(req_specs.modelling_wl);

In [ ]:
error_req, F_req = calc_rel_error(req_specs, x_req, solarIrr_req, refl_req, scenario.sza, σ_matrix_req, profile_req, h_req, ins_req, Sₐ_req, return_F=true) 
error_cbe, F_cbe = calc_rel_error(cbe_specs, x_cbe, solarIrr_cbe, refl_req, scenario.sza, σ_matrix_cbe, profile_cbe, h_cbe, ins_cbe, Sₐ_cbe, return_F=true) 

In [ ]:
@show error_cbe["co"]

In [ ]:
# Recompute for CBE but at 50m, i.e. 57ms
ins = InstrumentOperator.createGratingNoiseModel(57.0u"ms", cbe_specs.Pitch, cbe_specs.FPA_quantum_efficiency, cbe_specs.bench_efficiency, 
cbe_specs.Fnumber, 2*cbe_specs.SSI, (cbe_specs.readout_noise), cbe_specs.dark_current); 
error_cbe_50, F_cbe = calc_rel_error(cbe_specs, x_cbe, solarIrr_cbe, refl_req, scenario.sza, σ_matrix_cbe, profile_cbe, h_cbe, ins, Sₐ_cbe, return_F=true) 
@show error_cbe_50["n2o"]/background_vmr_n2o

In [ ]:
refRadiance = 1.0e16 # in photons/s/m²/nm
from_w_to_photons = 5.03e12
f = Figure(resolution=(550,400))
ax = Axis(f[1,1], xlabel="Wavelength (nm)", ylabel="Reflected Radiance (10¹⁶ photons/s/m²/nm)",  title="Reference Scenario ")
lines!(ax, cbe_specs.instrument_wl,F_cbe * from_w_to_photons .* cbe_specs.instrument_wl/1e16, color=CarbonI_colors[1], linewidth=2, label="CBE")
lines!(ax, req_specs.instrument_wl,F_req * from_w_to_photons .* req_specs.instrument_wl/1e16, color=CarbonI_colors[3], linewidth=1, label="Requirement")
lines!(ax, [2035,2375], [refRadiance/1e16,refRadiance/1e16], color=:black, linestyle=:dash, linewidth=1, label="Reference Radiance")
xlims!(ax, 2033, 2378)
axislegend(ax,position = :lb)
save("../../plots/final/Level1-4-BaselineScenario.pdf",f)
f

In [ ]:
# Compute Electrons at FPA
e_cbe = InstrumentOperator.photons_at_fpa(ins_cbe, (cbe_specs.instrument_wl)u"nm", (F_cbe)u"mW/m^2/nm/sr");
e_req = InstrumentOperator.photons_at_fpa(ins_req, (req_specs.instrument_wl)u"nm", (F_req)u"mW/m^2/nm/sr");
from_w_to_photons = 5.03e12
refRadiance = 1.0e16 # in photons/s/m²/nm

f = Figure(resolution=(550,400))
ax = Axis(f[1,1], xlabel="Wavelength (nm)", ylabel="Electrons measured at FPA",  title="Baseline Scenario (Flat Albedo=15%) ")
Label(f[1, 1, Top()], halign = :left, "x1000")
lines!(ax, cbe_specs.instrument_wl,e_cbe/1000, color=CarbonI_colors[1], linewidth=1.5, label="CBE")
lines!(ax, req_specs.instrument_wl,e_req/1000, color=CarbonI_colors[3], linewidth=1.5, label="Requirement")
#lines!(ax, [2035,2375], [refRadiance/ 1e16,refRadiance/ 1e16], color=:black, linestyle=:dash, linewidth=1, label="Reference Tropical Radiance")
xlims!(ax, 2033, 2378)
axislegend(ax,position = :lb)
ylims!(ax, 0, 60)
save("../../plots/final/Level1-4-BaselineScenario_FPA.pdf",f)
f

In [ ]:
# Save Data for plotting
allData_fpa_cbe = [cbe_specs.instrument_wl e_cbe];
allData_fpa_req = [req_specs.instrument_wl e_req];
writedlm("../../data/L1s/Level1-4-BaselineScenario_FPA_CBE.csv", allData_fpa_cbe);
writedlm("../../data/L1s/Level1-4-BaselineScenario_FPA_Req.csv", allData_fpa_req);

In [ ]:
# Now vary total efficiency at two different readout noise levels:

TotalEfficiencies = 0.3:0.05:1.0
n2o_errors_100 = zeros(length(TotalEfficiencies))
n2o_errors_120 = zeros(length(TotalEfficiencies))
for (i,efficiency) in enumerate(TotalEfficiencies)
    # Create instrument Specs here:
    # (here we need to use 57ms integration time for req. as we count 50m pixels along track dimension)
    ins_req_100 = InstrumentOperator.createGratingNoiseModel(57.0u"ms", req_specs.Pitch, 1.0, efficiency, req_specs.Fnumber, 2*req_specs.SSI, (100.0), req_specs.dark_current); 
    ins_req_120 = InstrumentOperator.createGratingNoiseModel(57.0u"ms", req_specs.Pitch, 1.0, efficiency, req_specs.Fnumber, 2*req_specs.SSI, (120.0), req_specs.dark_current); 
    
    # Create an error for 100 and 120e- for the respective efficiency
    error_varied_100 = calc_rel_error(req_specs, x_req, solarIrr_req, refl_req, scenario.sza, σ_matrix_req, profile_req, h_req, ins_req_100, Sₐ_req) 
    error_varied_120 = calc_rel_error(req_specs, x_req, solarIrr_req, refl_req, scenario.sza, σ_matrix_req, profile_req, h_req, ins_req_120, Sₐ_req) 
    
    # Store Data (no co-adding applied here)
    n2o_errors_100[i] = error_varied_100["n2o"]/background_vmr_n2o
    n2o_errors_120[i] = error_varied_120["n2o"]/background_vmr_n2o
end

In [ ]:
# Create an interpolation function to get the required optical efficiency for a given readout noise and proxy error for N2O
# Annoying as it requires ascending order in the x-axis
efficiency_100 = LinearInterpolation(reverse(100(n2o_errors_100)), reverse(TotalEfficiencies), extrapolation_bc=Line());
efficiency_120 = LinearInterpolation(reverse(100(n2o_errors_120)), reverse(TotalEfficiencies), extrapolation_bc=Line());

In [ ]:
f = Figure(resolution=(550,400), backgroundcolor=:transparent)
lowerLim = 4.0
ax = Axis(f[1,1], xlabel="Total System Efficiency", ylabel="Precision Error in Proxy (%)",  title="From required proxy precision to total system efficiency")
lines!(ax, TotalEfficiencies, n2o_errors_100*100, color=CarbonI_colors[1], linewidth=2, label="Readout Noise = 100e⁻")
lines!(ax, TotalEfficiencies, n2o_errors_120*100, color=CarbonI_colors[3], linewidth=2, label="Readout Noise = 120e⁻")

req_error        = 100*sigma_rel_SingleSounding_req /ampl
req_error_cbeGSD = 100*sigma_rel_SingleSounding_cbe /ampl
req_eff = efficiency_100(req_error)
req_eff_120 = efficiency_120(req_error)
eff_cbe = cbe_specs.bench_efficiency * cbe_specs.FPA_quantum_efficiency

arrows!(ax, [0.0,],[req_error,],[req_eff-0.01,],[0,],arrowsize = 15, linewidth=3,color = CarbonI_colors[5])
arrows!(ax, [0.0,],[req_error,],[req_eff_120-0.01,],[0,],arrowsize = 15, linewidth=3, color = CarbonI_colors[5])
arrows!(ax, [req_eff_120,],[req_error,],[eff_cbe-req_eff_120-0.005,],[0,],arrowsize = 15, linewidth=3, color = :black)
text!(ax, "Margin", position = (0.5, req_error),  align = (:center, :bottom), color = :black)
#arrows!(ax, [req_eff,],[req_error,],[0,],[-req_error+lowerLim + 0.05,],arrowsize = 10, linewidth=2,color = CarbonI_colors[5])
##text!(ax, "Req: > $(round(Int, 100req_eff))%", position = (req_eff, 5.5),
 #   align = (:center, :bottom), color = :black, rotation=π/2)

arrows!(ax, [req_eff_120,],[req_error,],[0,],[-req_error+lowerLim+0.15,],arrowsize = 15, linewidth=3, color = CarbonI_colors[5])
text!(ax, "Req: > $(round(Int, 100req_eff_120))%", position = (req_eff_120, 5.5),
    align = (:center, :bottom), color = :black, rotation=π/2)
lines!(ax, [eff_cbe,eff_cbe], [2,10], color=:black, linestyle=:dash, linewidth=3, label="CBE Total Efficiency")
scatter!(ax, [req_eff_120], [req_error], color=:red, markersize=15, label="Driving Req.")
#arrows!(ax2, [req_percent_ch4,],[req_flux_ch4,],[-req_percent_ch4+1,],[0,],arrowsize = 10, linewidth=2,color = CarbonI_colors[5])
#text!(ax2, "Req. >90%", position = (req_percent_ch4, 25),
#      align = (:center, :bottom), color = :black, rotation=π/2)

axislegend(ax,position = :rt)
xlims!(ax,0.3,0.8)
ylims!(ax,lowerLim,10)
save("../../plots/final/Level1-4-L1-5-reqs.pdf",f)
f

In [ ]:
# Save lines to merge later with other requirements:
AllData = [TotalEfficiencies n2o_errors_100*100 n2o_errors_120*100 ];

writedlm("../../data/L1s/L1-4_lines.dat", AllData)
writedlm("../../data/L1s/L1-4_req_error.dat", req_error)

In [ ]:
ins_req_forSNR = InstrumentOperator.createGratingNoiseModel(44.0u"ms", req_specs.Pitch, 1.0, req_eff_120, req_specs.Fnumber, 2*req_specs.SSI, (120.0), req_specs.dark_current);
ins_cbe_forSNR = InstrumentOperator.createGratingNoiseModel(44.0u"ms", cbe_specs.Pitch, 1.0, eff_cbe, cbe_specs.Fnumber, 2*cbe_specs.SSI, (100.0), cbe_specs.dark_current);
# Use reference Wavelength here (just for conversion)
ref_wl = 2250;

refRadiance = 1.0e16 # in photons/s/m²/nm
refRadiance_mw = refRadiance / from_w_to_photons / ref_wl
nesr_req_forSNR = InstrumentOperator.noise_equivalent_radiance(ins_req_forSNR, (ref_wl)u"nm", (refRadiance_mw)u"mW/m^2/nm/sr");
nesr_cbe_forSNR = InstrumentOperator.noise_equivalent_radiance(ins_cbe_forSNR, (ref_wl)u"nm", (refRadiance_mw)u"mW/m^2/nm/sr");

# @show refRadiance_mw/(nesr_req_forSNR[1]./1u"mW/m^2/nm/sr"), refRadiance_mw/(nesr_cbe_forSNR[1]./1u"mW/m^2/nm/sr")
@printf("Required SNR @1e16 photons/m2/sr/nm/s: %.4g\n",  refRadiance_mw/(nesr_req_forSNR[1]./1u"mW/m^2/nm/sr"))
@printf("CBE      SNR @1e16 photons/m2/sr/nm/s: %.4g",  refRadiance_mw/(nesr_cbe_forSNR[1]./1u"mW/m^2/nm/sr"))

In [ ]:
# Computing CBE for L1-5:
proxy_error = error_cbe["n2o"]/background_vmr_n2o
sigma_rel_cbe_agg = proxy_error / sqrt(N_eff_cbe)
@printf("CBE Standard Error for CH₄ : %.2g ppb\n", sigma_rel_cbe_agg*background_vmr_ch4) 
@printf("CBE Standard Error for CO₂ : %.2g ppm\n", sigma_rel_cbe_agg*background_vmr_co2)
@printf("CBE Standard Error for CO  : %.2g ppb\n", error_cbe["co"]/ sqrt(N_eff_cbe))

### The End